# Machine Translation

In [1]:
'''
Download dataset (originally from NIA AI-Hub)
'''

!gdown 1CpsqOuuuB3I_PG5DbuqH1ssCFVerU46g

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Downloading...
From: https://drive.google.com/uc?id=1CpsqOuuuB3I_PG5DbuqH1ssCFVerU46g
To: /home/teo/userdata/sg-aat-3020/aat3020-2022Spring/nia-aihub-korean-english.zip
100%|████████████████████████████████████████| 276M/276M [00:34<00:00, 8.06MB/s]


In [4]:
!unzip -q nia-aihub-korean-english.zip

In [6]:
from pathlib import Path

dataset_dir = Path('nia_korean_english')
data_list = sorted(list(dataset_dir.glob('*.xlsx')))
for path in dataset_dir.glob('*.xlsx'):
  df = pd.read_excel(path)
  kor_text_path = path.parent / (path.stem+'_kor.txt') 
  eng_text_path = path.parent / (path.stem+'_eng.txt') 
  with open(kor_text_path, 'w', encoding='utf8') as f:
      f.write('\n'.join(df['원문']))
  with open(eng_text_path, 'w', encoding='utf8') as f:
      f.write('\n'.join(df['번역문']))

/home/teo/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/teo/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/teo/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/teo/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
data_list

[PosixPath('nia_korean_english/1_구어체(1).xlsx'),
 PosixPath('nia_korean_english/1_구어체(2).xlsx'),
 PosixPath('nia_korean_english/2_대화체.xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(1)_200226.xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(2).xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(3).xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(4).xlsx'),
 PosixPath('nia_korean_english/4_문어체_한국문화.xlsx'),
 PosixPath('nia_korean_english/5_문어체_조례.xlsx'),
 PosixPath('nia_korean_english/6_문어체_지자체웹사이트.xlsx')]

In [8]:
import pandas as pd

dfs = [pd.read_excel(path) for path in data_list[:2]]

/home/teo/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
df = pd.concat(dfs, axis=0)

In [18]:
df['번역문'][10000:10050]

10000    Dogs, like dolphins and apes and parrots, can ...
10001    If the dog still keeps on hiccuping, then it m...
10002                                The dog will love it.
10003                                He's a beautiful dog.
10004                  I'm going to clean up the dog mess.
10005    I just woke up from my luggage because of the ...
10006    Knowing the meanings of the behaviors of pet d...
10007    When individuals stay closely together, they c...
10008    It is difficult to predict when each individua...
10009    I believe a true society can be achieved only ...
10010    For the first time in 100years since its incep...
10011    What did frog do after knowing that a snake ea...
10012     He turned into a frog and was bullied by people.
10013    The character is a naughty personality who is ...
10014    I thought he was always going get in trouble, ...
10015    There are fireworks on the national foundation...
10016    Not anyone or anything, look what you want to .

In [21]:
# path = data_list[0]

# kor_text_path = path.parent / (path.stem+'_kor.txt') 
# eng_text_path = path.parent / (path.stem+'_eng.txt') 
# with open(kor_text_path, 'w', encoding='utf8') as f:
#     f.write('\n'.join(df['원문']))
# with open(eng_text_path, 'w', encoding='utf8') as f:
#     f.write('\n'.join(df['번역문']))

## Huggingface Tokenizer

In [25]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)

# corpus_file   =  [str(path.parent / (path.stem + '_kor.txt')) for path in data_list[:2]]
# output_dir   = Path('hugging_kor_%d'%(vocab_size))
corpus_file   =  [str(path.parent / (path.stem + '_eng.txt')) for path in data_list[:2]]
output_dir   = Path('hugging_eng_%d'%(vocab_size))

vocab_size    = 32000  # Number of maximum size of the vocabulary
limit_alphabet= 6000   
output_dir   = Path('hugging_eng_%d'%(vocab_size))
output_dir.mkdir(exist_ok=True)
min_frequency = 5 

tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet, 
               show_progress=True)

tokenizer.save_model(str(output_dir))


['hugging_eng_32000/vocab.txt']

In [26]:
from transformers import BertTokenizerFast

tokenizer_src = BertTokenizerFast.from_pretrained('hugging_kor_32000',
                                                       strip_accents=False,
                                                       lowercase=False) 
tokenizer_tgt = BertTokenizerFast.from_pretrained('hugging_eng_32000',
                                                       strip_accents=False,
                                                       lowercase=False) 

tokenized_data = tokenizer_src(df['원문'].iloc[10])
print(tokenizer_src.decode(tokenized_data['input_ids']))

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


[CLS] 나는 친구에게 그 철학자의 책을 선물해 주겠다고 말했습니다. [SEP]


In [33]:
tokenized_ids = tokenizer_src("나는 학교에 갑니다")['input_ids']
tokenized_ids

[2, 3390, 4172, 4347, 3]

In [32]:
tokenizer_src.decode(tokenized_ids)

'[CLS] 나는 학교에 갑니다 [SEP]'

In [35]:
df.iloc[4]

SID                                                    5
원문        6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.
번역문    I would feel grateful to know how many stocks ...
Name: 4, dtype: object

## Divide Train / Validate/ Test Set
- using `np.random.choice`
    - To always get the same random shuffling result, you have to use `np.random.seed()`

## Define Dataset
- Each datasample has to return source sentence and target sentence
- You need a Tokenizer to get the tokenized result


In [39]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

## Define Collate function
- After implementing Dataset, we have to declare DataLoader that groups several training samples as a single batch
- However, we cannot batchify the melodies in straightforward way, because the length of each melody is different
- In this problem, you will learn about how to handle sequences of different length as a batch


In [ ]:

'''
This cell will make error, because the length of each sample is different to each other
'''

train_loader = DataLoader(train_set, batch_size=8)
batch = next(iter(train_loader))


In [ ]:
'''
To handle that problem, you have to make your collate function 
'''
def your_collate_function(raw_batch):
  '''
  You can make your own function to handle the batch
  '''
  
  return raw_batch[0] # This returns the first melody of each batch. So it will avoid the error, but it doesn't do proper batchifying

batch_size = 8
raw_batch = [train_set[i] for i in range(batch_size)] # This is the input for the collate function
batch = your_collate_function(raw_batch)

'''
This is what the 'collate_fn' does in DataLoader
'''

train_loader = DataLoader(train_set, batch_size=batch_size, collate_fn=your_collate_function)
batch_by_loader = next(iter(train_loader))

#### Pad Sequence and Pack Sequence
In PyTorch, there are two ways to batchify a group of sequence with different length.
- `torch.nn.utils.rnn.pad_sequence`
    - This function takes list of tensors with different length and return padded sequence
    - Padding is adding some constant number as a PAD token to match the length of short sequence to the maximum length
        - e.g. If there are sequence of length (3,7,4), we can add 4 zeros to sequence with length 3, 3 zeros to sequence with length 4 to make them length 7
    - In default, we use 0 for padding (zero padding)
    - The result 
- `torch.nn.utils.rnn.pack_sequence`
    - pad_sequence 

Cells below show the example of `pad_sequence`

In [41]:
from torch.nn.utils.rnn import pad_sequence, pack_sequence, PackedSequence
short = torch.arange(3, -1, -1).float() # [3, 2, 1, 0]
long = torch.arange(27,19, -1).float()
middle = torch.arange(15,9, -1).float()

pad_sequence([short, long, middle], batch_first=False)  # T x N 

tensor([[ 3., 27., 15.],
        [ 2., 26., 14.],
        [ 1., 25., 13.],
        [ 0., 24., 12.],
        [ 0., 23., 11.],
        [ 0., 22., 10.],
        [ 0., 21.,  0.],
        [ 0., 20.,  0.]])

In [ ]:
# Default value of batch_first in pad_sequence is False.
# So you have to always be careful not to miss batch_first=True in pad_sequence, if you use batch_first=True for your RNN layer.
pad_sequence([short, long, middle], batch_first=True)  # N x T 

1) However, the problem is that you can't figure out whether the 0 at the end of each sequence is a padded one, or was included in the original sequence
- e.g. `[2, 3, 4, 3, 0]` becomes `[ 2,  3,  4,  5,  0,  0,  0]`. Now we don't know how many zeros were added for padding

2) Also, if you run RNN for this padded sequence, RNN will calculate for the padded part also.
- RNN doesn't know whether it is padded data, or existing data
- This makes computation slower

3) If you want to use bi-directional, which also reads the sequence from backward, paddings can make the result different.

To solve this issue, we use PackedSequence, by using `pack_sequence`/

In [36]:
packed_sequence = pack_sequence([short, long, middle], enforce_sorted=False)
packed_sequence

PackedSequence(data=tensor([27, 15,  3, 26, 14,  2, 25, 13,  1, 24, 12,  0, 23, 11, 22, 10, 21, 20]), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

`PackedSequence` has `data` and `batch_sizes`
- `data` contains the flattened value of given batch
    - To optimize the computation, the sequences have to be sorted by descending of length
- `batch_sizes` represents how many valid batch sample exists for each time step
    - `[3, 3, 3, 2, 2, 1, 1]` means that there are 3 sequences for first three time steps, and then 2 sequences for next two steps, and then only 1 sequence for next two steps.
- `sorted_indices` shows how the sorted sequences can be converted to original order.
    - `[1,2,0]` means that 
        - the 0th sequence in the sorted sequences (the longest one) was indexed as 1 in the original input batch
        - the 1st sequence in the sorted sequences (`middle`) was indexed as 2 in the original input batch
        - the 2nd sequence in the sorted sequences (`short`) was index as 0 in the original input batch
- `unsorted_indices` shows how the original sequences are sorted.
    - `[2,0,1]` means that
        - the 0th sequence in the original input was sorted as 2nd in the sorted sequences

In [42]:
rnn_layer = nn.GRU(1, 1)
packed_sequence = pack_sequence([short.unsqueeze(1), long.unsqueeze(1), middle.unsqueeze(1)], enforce_sorted=False)
out, last_hidden = rnn_layer(packed_sequence)

print(f"Type of output of RNN for PackedSequence: {type(out)}")
print(f"Type of last_hidden of RNN for PackedSequence: {type(last_hidden)}")

Type of output of RNN for PackedSequence: <class 'torch.nn.utils.rnn.PackedSequence'>
Type of last_hidden of RNN for PackedSequence: <class 'torch.Tensor'>


- RNN or its family of PyTorch can automatically handle `PackedSequence`
- However, other layers like `nn.Embedding` or `nn.Linear` cannot take `PackedSequence` as its input
- There are two ways to feed `PackedSequence` to these layers
    - First, convert PackedSequence to ordinary torch.Tensor by `torch.nn.utils.rnn.pad_packed_sequence`
        - This will convert PackedSequence to a tensor of sequneces with same length but different padding
    - The other way is to feed only PackedSequence.data, and then declaring new PackedSequence with the output as `data`

In [ ]:
'''
This will make error, because other layers cannot handle PackedSequence
'''
test_linear_layer = nn.Linear(in_features=1, out_features=2)
test_linear_layer(packed_sequence)

In [ ]:
'''
One way to to this is using torch.nn.utils.rnn.pad_packed_sequence to convert PackedSequence to ordinary tensor
'''

from torch.nn.utils.rnn import pad_packed_sequence
padded_sequence, batch_lengths = pad_packed_sequence(packed_sequence)
print(f'The padded sequence generated from packed sequence (squeezed for printing): \n {padded_sequence.squeeze()}')
print(f'"pad_packed_sequence" also returns "batch_lengths", to clarify the original length before the padding: \n {batch_lengths}')



In [ ]:
'''
Now you can feed padded sequence to linear layer.
'''

linear_output = test_linear_layer(padded_sequence)
print(f"Output of feeding padded_sequence to a linear layer: {linear_output}")
print("Caution that it returns non-zero values for timestep with zero padding, because linear layer has a bias")

In [ ]:
'''
You can make the output as a PackedSequence, by using torch.nn.utils.rnn.pack_padded_sequence
'''
from torch.nn.utils.rnn import pack_padded_sequence
re_packed_sequence = pack_padded_sequence(linear_output, batch_lengths, enforce_sorted=False)
re_packed_sequence

In [ ]:
'''
Another way to do it is using PackedSequence.data
'''

linear_out_pack = test_linear_layer(packed_sequence.data)
packed_sequence_after_linear = PackedSequence(linear_out_pack, packed_sequence.batch_sizes, packed_sequence.sorted_indices, packed_sequence.unsorted_indices)
packed_sequence_after_linear

## Define Model
![image](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/seq2seq.jpg)

## Define Trainer

In [ ]:
from tqdm import tqdm

class Trainer:
  def __init__(self, model, optimizer, loss_fn, train_loader, valid_loader, device):
    self.model = model
    self.optimizer = optimizer
    self.loss_fn = loss_fn
    self.train_loader = train_loader
    self.valid_loader = valid_loader
    
    self.model.to(device)
    
    self.best_valid_accuracy = 0
    self.device = device
    
    self.training_loss = []
    self.validation_loss = []
    self.validation_acc = []

  def save_model(self, path='imdb_sentiment_model.pt'):
    torch.save({'model':self.model.state_dict(), 'optim':self.optimizer.state_dict()}, path)
    
  def train_by_num_epoch(self, num_epochs):
    for epoch in tqdm(range(num_epochs)):
      self.model.train()
      for batch in self.train_loader:
        loss_value = self._train_by_single_batch(batch)
        self.training_loss.append(loss_value)
      self.model.eval()
      validation_loss, validation_acc = self.validate()
      self.validation_loss.append(validation_loss)
      self.validation_acc.append(validation_acc)
      
      if validation_acc > self.best_valid_accuracy:
        print(f"Saving the model with best validation accuracy: Epoch {epoch+1}, Acc: {validation_acc:.4f} ")
        self.save_model('imdb_sentiment_model_best.pt')
      else:
        self.save_model('imdb_sentiment_model_last.pt')
      self.best_valid_accuracy = max(validation_acc, self.best_valid_accuracy)

      
  def _train_by_single_batch(self, batch):
    '''
    This method updates self.model's parameter with a given batch
    
    batch (tuple): (batch_of_input_text, batch_of_label)
    
    You have to use variables below:
    
    self.model (SentimentModel/torch.nn.Module): A neural network model
    self.optimizer (torch.optim.adam.Adam): Adam optimizer that optimizes model's parameter
    self.loss_fn (function): function for calculating BCE loss for a given prediction and target
    self.device (str): 'cuda' or 'cpu'

    output: loss (float): Mean binary cross entropy value for every sample in the training batch
    The model's parameters, optimizer's steps has to be updated inside this method

    TODO: Complete this method 
    '''

    
    return

    
  def validate(self, external_loader=None):
    '''
    This method calculates accuracy and loss for given data loader.
    It can be used for validation step, or to get test set result
    
    input:
      data_loader: If there is no data_loader given, use self.valid_loader as default.
      
    
    output: 
      validation_loss (float): Mean Binary Cross Entropy value for every sample in validation set
      validation_accuracy (float): Mean Accuracy value for every sample in validation set
      
    TODO: Complete this method 

    '''
    
    ### Don't change this part
    if external_loader and isinstance(external_loader, DataLoader):
      loader = external_loader
      print('An arbitrary loader is used instead of Validation loader')
    else:
      loader = self.valid_loader
      
    self.model.eval()
    
    '''
    Write your code from here, using loader, self.model, self.loss_fn.
    '''